In [ ]:
!pip -q install "camel-ai[all]" "python-dotenv" "rich"

import os
import json
import time
from typing import Dict, Any
from rich import print as rprint

def load_openai_key() -> str:
    key = None
    try:
        from google.colab import userdata
        key = userdata.get("OPENAI_API_KEY")
    except Exception:
        key = None
    if not key:
        import getpass
        key = getpass.getpass("Enter OPENAI_API_KEY (hidden): ").strip()
    if not key:
        raise ValueError("OPENAI_API_KEY is required.")
    return key

os.environ["OPENAI_API_KEY"] = load_openai_key()

In [ ]:
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType
from camel.agents import ChatAgent
from camel.toolkits import SearchToolkit

MODEL_CFG = {"temperature": 0.2}

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O,
    model_config_dict=MODEL_CFG,
)

In [ ]:
MEM_PATH = "camel_memory.json"

def mem_load() -> Dict[str, Any]:
    if not os.path.exists(MEM_PATH):
        return {"runs": []}
    with open(MEM_PATH, "r", encoding="utf-8") as f:
        return json.load(f)

def mem_save(mem: Dict[str, Any]) -> None:
    with open(MEM_PATH, "w", encoding="utf-8") as f:
        json.dump(mem, f, ensure_ascii=False, indent=2)

def mem_add_run(topic: str, artifacts: Dict[str, str]) -> None:
    mem = mem_load()
    mem["runs"].append({"ts": int(time.time()), "topic": topic, "artifacts": artifacts})
    mem_save(mem)

def mem_last_summaries(n: int = 3) -> str:
    mem = mem_load()
    runs = mem.get("runs", [])[-n:]
    if not runs:
        return "No past runs."
    return "\n".join([f"{i+1}. topic={r['topic']} | ts={r['ts']}" for i, r in enumerate(runs)])

In [ ]:
def make_agent(role: str, goal: str, extra_rules: str = "") -> ChatAgent:
    system = (
        f"You are {role}.\n"
        f"Goal: {goal}\n"
        f"{extra_rules}\n"
        "Output must be crisp, structured, and directly usable by the next agent."
    )
    return ChatAgent(model=model, system_message=system)

planner = make_agent(
    "Planner",
    "Create a compact plan and research questions with acceptance criteria.",
    "Return JSON with keys: plan, questions, acceptance_criteria."
)

researcher = make_agent(
    "Researcher",
    "Answer questions using web search results.",
    "Return JSON with keys: findings, sources, open_questions."
)

writer = make_agent(
    "Writer",
    "Draft a structured research brief.",
    "Return Markdown only."
)

critic = make_agent(
    "Critic",
    "Identify weaknesses and suggest fixes.",
    "Return JSON with keys: issues, fixes, rewrite_instructions."
)

finalizer = make_agent(
    "Finalizer",
    "Produce the final improved brief.",
    "Return Markdown only."
)

search_tool = SearchToolkit().search_duckduckgo
researcher = ChatAgent(
    model=model,
    system_message=researcher.system_message,
    tools=[search_tool],
)

In [ ]:
def step_json(agent: ChatAgent, prompt: str) -> Dict[str, Any]:
    res = agent.step(prompt)
    txt = res.msgs[0].content.strip()
    try:
        return json.loads(txt)
    except Exception:
        return {"raw": txt}

def step_text(agent: ChatAgent, prompt: str) -> str:
    res = agent.step(prompt)
    return res.msgs[0].content

In [4]:
def run_workflow(topic: str) -> Dict[str, str]:
    rprint(mem_last_summaries(3))

    plan = step_json(
        planner,
        f"Topic: {topic}\nCreate a tight plan and research questions."
    )

    research = step_json(
        researcher,
        f"Research the topic using web search.\n{json.dumps(plan)}"
    )

    draft = step_text(
        writer,
        f"Write a research brief using:\n{json.dumps(research)}"
    )

    critique = step_json(
        critic,
        f"Critique the draft:\n{draft}"
    )

    final = step_text(
        finalizer,
        f"Rewrite using critique:\n{json.dumps(critique)}\nDraft:\n{draft}"
    )

    artifacts = {
        "plan_json": json.dumps(plan, indent=2),
        "research_json": json.dumps(research, indent=2),
        "draft_md": draft,
        "critique_json": json.dumps(critique, indent=2),
        "final_md": final,
    }

    mem_add_run(topic, artifacts)
    return artifacts

TOPIC = "Agentic multi-agent research workflow with quality control"
artifacts = run_workflow(TOPIC)
print(artifacts["final_md"])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.1/52.1 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.6/91.6 kB 6.8 MB/s eta 0:00:00
  Pre

Past runs:

No past runs.

1) Planner -> plan + questions

{
    'raw': '```json\n{\n  "plan": [\n    "Define the scope and objectives of the research assistant workflow.",\n  
"Identify key functionalities and features required for the research assistant.",\n    "Research existing 
technologies and tools that can be integrated for quality control and hallucination prevention.",\n    "Design a 
workflow architecture that incorporates these technologies and tools.",\n    "Develop guardrails to prevent 
hallucination, such as fact-checking mechanisms and source verification.",\n    "Implement quality control 
measures, including peer review and feedback loops.",\n    "Test the workflow with a sample dataset to evaluate 
performance and identify areas for improvement.",\n    "Iterate on the design based on testing feedback and refine 
the workflow.",\n    "Document the workflow process, including guardrails and quality control measures."\n  ],\n  
"questions": [\n    "What are the common causes of hallucination in AI systems, and how can they be mitigated?",\n 
"What existing tools and technologies are available for quality control in AI workflows?",\n    "How can 
fact-checking mechanisms be effectively integrated into an AI research assistant?",\n    "What are the best 
practices for designing guardrails in AI systems?",\n    "How can user feedback be incorporated into the workflow 
to improve accuracy and reliability?",\n    "What metrics should be used to evaluate the performance of the 
research assistant workflow?"\n  ],\n  "acceptance_criteria": [\n    "The workflow must clearly define its scope 
and objectives.",\n    "Key functionalities and features must be identified and documented.",\n    "At least three 
existing technologies or tools must be researched and considered for integration.",\n    "The workflow architecture
must include specific guardrails for hallucination prevention.",\n    "Quality control measures must be clearly 
outlined and implemented.",\n    "The workflow must be tested with a sample dataset, and results must be 
documented.",\n    "Feedback from testing must be used to iterate and improve the workflow.",\n    "Comprehensive 
documentation of the workflow, including guardrails and quality control measures, must be provided."\n  ]\n}\n```'
}

...

# Addressing AI System Challenges: Hallucinations, Quality Control, and User Feedback

## Executive Summary
- Encountered rate limits while accessing DuckDuckGo for research on AI system challenges.
- Key areas of interest included AI hallucinations, quality control, and fact-checking mechanisms.
- Sought best practices for designing guardrails and incorporating user feedback in AI systems.
- Attempted to identify metrics for evaluating AI research assistant workflows.
- Recommended alternative search methods due to access issues.

## Introduction
The research aimed to explore critical challenges in AI systems, focusing on hallucinations, quality control, and 
user feedback integration. However, access to DuckDuckGo was restricted due to rate limits, necessitating 
alternative approaches to gather the required information.

## Common Causes of Hallucination in AI Systems and Mitigation Strategies
Understanding hallucinations in AI systems is crucial for improving accuracy and reliability. Hallucinations occur 
when AI generates outputs that are not grounded in the input data, often due to model overfitting or inadequate 
training data. Mitigation strategies include refining training da
...

4) Critic -> issues + rewrite instructions

{
    'raw': '```json\n{\n  "issues": [\n    "The draft lacks specific examples or case studies to illustrate the 
challenges and solutions discussed.",\n    "There is no mention of the ethical implications of AI hallucinations 
and quality control failures.",\n    "The draft does not address the potential limitations and biases of 
fact-checking mechanisms.",\n    "The section on user feedback integration lacks depth on how feedback is analyzed 
and acted upon.",\n    "The risks/limitations section is brief and does not explore the broader implications of the
identified risks."\n  ],\n  "fixes": [\n    "Include specific examples or case studies of AI systems experiencing 
hallucinations and how they were addressed.",\n    "Discuss the ethical considerations related to AI hallucinations
and quality control, such as the impact on decision-making and user trust.",\n    "Expand on the limitations and 
potential biases of fact-checking mechanisms, including how they can be mitigated.",\n    "Provide more detail on 
the methods for analyzing user feedback and how it informs system improvements.",\n    "Enhance the 
risks/limitations section by exploring the broader implications of rate limits and data access issues on AI 
research."\n  ],\n  "rewrite_instructions": [\n    "Add case studies or examples to illustrate the challenges and 
solutions in each section.",\n    "Incorporate a discussion on the ethical implications of AI system failures, 
particularly hallucinations and quality control.",\n    "Expand the section on fact-checking mechanisms to include 
potential biases and limitations.",\n    "Provide a more detailed explanation of how user feedback is collected, 
analyzed, and used to improve AI systems.",\n    "Broaden the risks/limitations section to include a discussion on 
the impact of data access issues on AI research and development."\n  ]\n}\n```'
}

5) Finalizer -> improved final brief



================ FINAL BRIEF ================

# Addressing AI System Challenges: Hallucinations, Quality Control, and User Feedback

## Executive Summary
This brief explores critical challenges in AI systems, focusing on hallucinations, quality control, and user feedback integration. Due to rate limits on DuckDuckGo, alternative methods are recommended for gathering information.

## Introduction
The research aims to address key issues in AI systems, including hallucinations, quality control, and user feedback. Access to DuckDuckGo was restricted, highlighting the need for alternative data sources.

## Common Causes of Hallucination in AI Systems and Mitigation Strategies
AI hallucinations occur when outputs are not grounded in input data, often due to overfitting or inadequate training. Mitigation includes refining datasets, robust validation, and real-time feedback loops. Case studies, such as OpenAI's GPT-3, illustrate the importance of these strategies.

## Existing Tools and Tec